In [1]:
import os

import pandas as pd
import pyproj

# Converting between vertical datums

SeaBee pilots need to be able to convert between vertical datums as well as perform 2D co-ordinate conversions. This notebook explores options using [PyProj](https://pyproj4.github.io/pyproj/stable/index.html).

## 1. Background

CRSs may be 2D or 3D. 3D systems use a 2D CRS for (x, y), plus an additional axis for elevations. For example, [EPSG 4326](https://epsg.org/crs_4326/WGS-84.html) is a 2D geographic CRS measured relative to the WGS84 ellipsoid, whereas [EPSG 4979](https://epsg.org/crs_4979/WGS-84.html) is the 3D equivalent, where x and y are longitude and latitude (just like for EPSG 4326) and z is elevation in metres measured relative to the WGS84 ellipsoid. 

It is also possible to have projected co-ordinates for x and y, with elevations measured relative to various underlying height models. For example, [EPSG 25833](https://epsg.org/crs_25833/ETRS89-UTM-zone-33N.html) is a 2D projected system (UTM Zone 33N) relative to the ETRS89 datum, while [EPSG 5973](https://epsg.org/crs_5973/ETRS89-UTM-zone-33N-NN2000-height.html) is a 3D compound CRS combining ETRS89 UTM Zone 33N with the NN2000 height model (which is the most up-to-date height model for Norway).

[This](https://register.geonorge.no/epsg-koder) website provides EPSG codes for various CRSs relevant to Norway. [This](https://gis.stackexchange.com/a/385738/2131) post from Stack Exchange is also helpful.

## 2. Proj data

By default, Proj only installs the most commonly used transformation files. Sometimes it gives helpful errors when files are missing, but in some cases it fails "silently" i.e. the input co-ordinates are returned unchanged, but it otherwise everything seems to work. This problem can be solved by installing the necessary data files or, if disk space is not an issue, just installing everything using 

    mamba install -c conda-forge proj-data

## 3. Convert NN2000 vertical datum to WGS84

The code below assumes an input file containing `x` and `y` values in ETRS89-UTM co-ordinates and `z` values in metres expressed relative to NN2000. The `x` and `y` co-ordinates are left unchanged, while the `z` values are expressed relative to the WGS84 datum.

### 3.1. User input

In [2]:
utm_zone = 32
file_path = "~/convert_vertical_datum/202300505_gcps.csv"

### 3.2. Convert

In [3]:
# Identify input EPSG
valid_zones = range(31, 36)
assert utm_zone in valid_zones, "'utm_zone' must be an integer between 31 and 35."
input_crs = 5940 + utm_zone

# Read data
df = pd.read_csv(file_path, sep=";", names=["id", "x", "y", "z"])

# Setup projections
src_crs = pyproj.crs.CRS.from_epsg(input_crs)
wgs84_3d_crs = pyproj.crs.CRS.from_epsg(4979)

# Convert
transformer = pyproj.transformer.Transformer.from_crs(
    crs_from=src_crs, crs_to=wgs84_3d_crs, always_xy=True
)
x_wgs84, y_wgs84, df["z"] = transformer.transform(df.x, df.y, df.z)
df['z'] = df['z'].round(3)

# Save 
base_name, extension = os.path.splitext(file_path)
csv_path = base_name + "_converted" + extension
df.to_csv(csv_path, index=False, header=None, sep=";")

df.head()

,id,x,y,z
0,GS18T_1,565025.763,6541468.499,41.510
1,GS18T_2,565043.528,6541413.865,44.673
2,GS18T_3,565074.924,6541368.255,40.215
3,GS18T_4,565108.528,6541304.199,40.671
4,GS18T_5,564998.028,6541340.293,40.200
